In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [2]:
#load Data

file_name = "./data/train_preprocessed2.csv"
train_df = pd.read_csv(file_name, low_memory = False)

train_df.head()

,Unnamed: 0,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,...,Weekday,Month,Day.of.Month,Day.of.Year,Season,Sponsor.Code,Grant.Category.Code,Contract.Value.Band,RFCD.Code,SEO.Code
0,0,4.0,2.0,0.0,0.0,1,0.00,1,0.0,0.0,...,3,8,11,223,3,0.0,0.0,50000,"[4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,1,6.0,12.0,2.0,2.0,1,1.00,1,1.0,20.0,...,4,11,11,315,4,2.0,1.0,100000,"[2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ..."
2,2,7.0,20.0,20.0,7.0,2,0.75,4,2.0,50.0,...,0,11,14,318,4,29.0,5.0,50000,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, ...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
3,3,0.0,3.0,13.0,3.0,1,1.00,2,2.0,15.0,...,1,11,15,319,4,40.0,5.0,200000,"[3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
4,4,3.0,0.0,1.0,0.0,1,0.00,1,1.0,10.0,...,2,11,16,320,4,59.0,1.0,50000,"[2, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
train_df.drop('Unnamed: 0', axis=1, inplace = True)
train_df.head()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,1,0.0,0.0,1.0,280199.0,100.0,0.0,0.0,0.0,0.0,...,3073.0,31.0,0.0,1.0,0.0,0.0,4.0,2.0,0.0,0.0
1,1,2.0,1.0,2.0,280103.0,30.0,280106.0,30.0,280203.0,40.0,...,2538.0,25.0,1.0,2.0,0.0,0.0,6.0,12.0,2.0,2.0
2,1,29.0,2.0,1.0,321004.0,60.0,321216.0,40.0,0.0,0.0,...,2923.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,5.0,2.0
3,1,40.0,2.0,3.0,270602.0,50.0,320602.0,50.0,0.0,0.0,...,2678.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,13.0,3.0
4,0,59.0,1.0,1.0,260500.0,34.0,280000.0,33.0,290000.0,33.0,...,2153.0,19.0,1.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0


In [4]:
#Setup data

array = train_df.values

data = array[:, 1:28]
target = array[:, 0]

data, target

seed = 7
test_size = 0.2

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = test_size, random_state = seed)

In [5]:
#Setup XGBoost Model

model = XGBClassifier()
model.fit(data_train, target_train)
print(model)

ValueError: could not convert string to float: '[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]'

In [19]:
#Make predictions with XGBoost Model
target_pred = model.predict(data_test)
predictions = [round(value) for value in target_pred]
accuracy = accuracy_score(target_test, predictions)
print("Accuracy : %.2f%%" %(accuracy * 100))


Accuracy : 85.30%


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
#Create Pipeline

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('normalize', Normalizer()))
estimators.append(('lda', LinearDiscriminantAnalysis()))
model = Pipeline(estimators)

In [6]:
#Evaluate Pipeline

seed = 7
kfold = KFold(n_splits = 10, random_state = seed)
results = cross_val_score(model, data, target, cv = kfold)
print(results.mean())

0.7255427700514384


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Library/F